In [7]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [25]:
def create_training_data_chat(file, n_mels, n_fft, hop_length, onset_list):
    # Load the audio file
    print(file)
    y, sr = librosa.load(file)
    print(sr)
    
    # Compute the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    num_frames = mel_spectrogram.shape[1]
    
    # Determine the time per frame
    frame_duration = hop_length / sr
    
    # Initialize empty DataFrame
    start_end_spect_target = pd.DataFrame(columns=['Start sample', 'End sample', 'Spectogram', 'onset'])
    
    if n_fft == hop_length:
        # Case where n_fft equals hop_length
        start_samples = np.arange(num_frames) * frame_duration
        end_samples = start_samples + frame_duration
        
        start_end_spect_target = pd.DataFrame({
            'Start sample': start_samples,
            'End sample': end_samples,
            'Spectogram': [mel_spectrogram[:, i] for i in range(num_frames)],
            'onset': 0
        })
    else:
        # Case where n_fft is not equal to hop_length
        start_samples = np.arange(1, num_frames) * frame_duration
        end_samples = start_samples + frame_duration
        
        diffs = [mel_spectrogram[:, i - 1] - mel_spectrogram[:, i] for i in range(1, num_frames)]
        
        start_end_spect_target = pd.DataFrame({
            'Start sample': start_samples,
            'End sample': end_samples,
            'Spectogram': diffs,
            'onset': 0
        })
    
    # Mark the onset frames based on the onset list
    for onset in onset_list:
        mask = (start_end_spect_target['Start sample'] <= onset) & (onset <= start_end_spect_target['End sample'])
        start_end_spect_target.loc[mask, 'onset'] = 1

    return start_end_spect_target


In [30]:
'''
    Do for every file:
        n_fft: 128, hop_length: 64
        n_fft: 128, hop_length: 128
        n_fft: 256, hop_length: 128
        n_fft: 256, hop_length: 256

'''
onset = pd.read_csv('../onsets_ISMIR_2012/annotations/onsets/ah_development_percussion_castagnet1.onsets',header=None)
onset = onset[0].values.tolist()
n_mels = 254
n_fft = 64
hop_length = 64
file_path = '../onsets_ISMIR_2012/audio/ah_development_percussion_castagnet1.flac'
test1 = create_training_data_chat(file_path,n_mels,n_fft,hop_length,onset)
test1

../onsets_ISMIR_2012/audio/ah_development_percussion_castagnet1.flac
22050


c:\Users\20192801\Anaconda3\envs\DeepLearning_labs\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


,Start sample,End sample,Spectogram,onset
0,0.000000,0.002902,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1,0.002902,0.005805,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
2,0.005805,0.008707,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3,0.008707,0.011610,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
4,0.011610,0.014512,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
...,...,...,...,...
3096,8.986122,8.989025,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3097,8.989025,8.991927,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3098,8.991927,8.994830,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3099,8.994830,8.997732,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0


In [36]:
import random

# Total number of files
total_files = 200

# Create a list of file indices from 1 to 200
file_indices = list(range(1, total_files + 1))

# Shuffle the list of file indices to ensure random order
random.shuffle(file_indices)

# Calculate the number of files for each dataset
num_files_dataset1 = int(0.60 * total_files)  # 60% of files
num_files_dataset2 = int(0.20 * total_files)  # 20% of files
# The last 20% can simply be the remainder of the list
num_files_dataset3 = total_files - num_files_dataset1 - num_files_dataset2

# Assign files to datasets
dataset1 = file_indices[:num_files_dataset1]
dataset2 = file_indices[num_files_dataset1:num_files_dataset1 + num_files_dataset2]
dataset3 = file_indices[num_files_dataset1 + num_files_dataset2:]

# Print the datasets (optional)
print("Dataset 1:", dataset1)
print("Dataset 2:", dataset2)
print("Dataset 3:", dataset3)


Dataset 1: [176, 175, 135, 27, 105, 109, 72, 123, 8, 71, 110, 55, 7, 64, 54, 90, 17, 80, 138, 181, 151, 148, 74, 18, 58, 171, 188, 166, 46, 196, 66, 117, 146, 106, 36, 40, 48, 163, 149, 184, 83, 82, 28, 200, 172, 2, 157, 122, 120, 79, 98, 189, 67, 154, 37, 52, 31, 127, 161, 15, 12, 60, 165, 102, 75, 155, 115, 1, 187, 132, 49, 158, 9, 6, 21, 50, 85, 152, 91, 92, 76, 29, 142, 53, 47, 3, 11, 167, 99, 111, 139, 133, 137, 199, 131, 164, 51, 194, 134, 42, 100, 197, 124, 177, 160, 56, 61, 114, 186, 153, 65, 23, 14, 183, 96, 125, 192, 173, 170, 32]
Dataset 2: [81, 119, 118, 5, 159, 150, 198, 84, 108, 103, 57, 104, 73, 107, 143, 190, 156, 128, 97, 94, 129, 113, 16, 185, 78, 147, 33, 168, 191, 121, 95, 162, 89, 144, 20, 38, 178, 35, 193, 24]
Dataset 3: [145, 70, 10, 44, 34, 116, 63, 136, 43, 112, 68, 62, 45, 195, 69, 86, 13, 140, 169, 130, 22, 39, 25, 88, 19, 93, 4, 30, 41, 101, 59, 141, 179, 26, 182, 126, 180, 174, 87, 77]


In [35]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import os

base_path = 'C:/Users/20192801/Documents/deep_learning_music_project'

annotations_dir = os.path.join(base_path, 'onsets_ISMIR_2012', 'annotations', 'onsets')
audio_dir = os.path.join(base_path, 'onsets_ISMIR_2012', 'audio')

# Retrieve all csv and audio files
list_of_onset_paths = [os.path.join(annotations_dir, f) for f in os.listdir(annotations_dir) if f.endswith('.onsets')]
list_of_flac_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith('.flac')]

n_fft = 512
hop_length = 256
number_mels = 254


def compute_and_save_spect_target_dataframes(list_of_csv_paths, list_of_wav_paths, your_path, hop_length, n_fft,
                                             number_mels):
    # Ensure the lengths of the lists are the same
    if len(list_of_csv_paths) != len(list_of_wav_paths):
        raise ValueError("The lengths of the CSV and WAV file path lists must be equal.")

    # Initialize a list to hold confirmation messages
    confirmation_messages = []

    # Iterate over each pair of CSV and WAV file paths
    for csv_path, wav_path in zip(list_of_csv_paths, list_of_wav_paths):
        # Extract the name of the file (e.g., 'Haslebuskane_happy' from the WAV file path)
        file_name = wav_path.split('/')[-1].replace('.flac', '')

        # Extract the name of the file without extension
        name = os.path.splitext(os.path.basename(wav_path))[0]

        # Load the annotations from the CSV file
        label_frame = pd.read_csv(csv_path, header=None)
        onset_list = label_frame[0].values.tolist()

        # Load the audio file
        y, sr = librosa.load(wav_path)
        
        

        # Compute the Mel spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length,
                                                         n_mels=number_mels)
        num_frames = mel_spectrogram.shape[1]

        # Initialize a DataFrame to hold the start, end samples, spectrogram, and onset data
        start_end_spect_target = pd.DataFrame(columns=['Start sample', 'End sample', 'Spectrogram', 'onset'])

        # Calculate sample time
        frame_duration = hop_length / sr

        # Prepare data frames for storing the spectrogram information
        start_samples = np.arange(num_frames) * frame_duration
        end_samples = start_samples + frame_duration

        # Create DataFrame
        start_end_spect_target = pd.DataFrame({
            'Start sample': start_samples,
            'End sample': end_samples,
            'Spectrogram': [mel_spectrogram[:, i] for i in range(num_frames)],
            'onset': 0
        })

        # Mark the onset frames based on the onset list
        for onset in onset_list:
            mask = (start_end_spect_target['Start sample']-0.025 <= onset) & (onset <= start_end_spect_target['End sample']+0.025)
            start_end_spect_target.loc[mask, 'onset'] = 1

        # Construct the path where each CSV will be saved
        csv_save_path = os.path.join(your_path, 'onsets_ISMIR_2012', 'csv_large_label',
                                     f"{name}_start_end_spect_target.csv")

        # Ensure the directory exists before saving
        os.makedirs(os.path.dirname(csv_save_path), exist_ok=True)
        print(csv_save_path)
        # Save the computed DataFrame as a CSV file
        start_end_spect_target.to_csv(csv_save_path, index=False)

        # Add a confirmation message to the list
        confirmation_messages.append(f"File '{csv_save_path}' saved successfully.")

    return confirmation_messages


compute_and_save_spect_target_dataframes(list_of_onset_paths, list_of_flac_files, base_path,n_fft,hop_length,number_mels)

c:\Users\20192801\Anaconda3\envs\DeepLearning_labs\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


C:/Users/20192801/Documents/deep_learning_music_project\onsets_ISMIR_2012\csv_large_label\ah_development_guitar_2684_TexasMusicForge_Dandelion_pt1_start_end_spect_target.csv


c:\Users\20192801\Anaconda3\envs\DeepLearning_labs\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


C:/Users/20192801/Documents/deep_learning_music_project\onsets_ISMIR_2012\csv_large_label\ah_development_guitar_Guitar_Licks_06-11_start_end_spect_target.csv


c:\Users\20192801\Anaconda3\envs\DeepLearning_labs\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


...

c:\Users\20192801\Anaconda3\envs\DeepLearning_labs\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


C:/Users/20192801/Documents/deep_learning_music_project\onsets_ISMIR_2012\csv_large_label\vorbis_dr4_start_end_spect_target.csv


c:\Users\20192801\Anaconda3\envs\DeepLearning_labs\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


C:/Users/20192801/Documents/deep_learning_music_project\onsets_ISMIR_2012\csv_large_label\vorbis_lalaw_start_end_spect_target.csv


["File 'C:/Users/20192801/Documents/deep_learning_music_project\\onsets_ISMIR_2012\\csv_large_label\\ah_development_guitar_2684_TexasMusicForge_Dandelion_pt1_start_end_spect_target.csv' saved successfully.",
 "File 'C:/Users/20192801/Documents/deep_learning_music_project\\onsets_ISMIR_2012\\csv_large_label\\ah_development_guitar_Guitar_Licks_06-11_start_end_spect_target.csv' saved successfully.",
 "File 'C:/Users/20192801/Documents/deep_learning_music_project\\onsets_ISMIR_2012\\csv_large_label\\ah_development_guitar_Guitar_Licks_06-12_start_end_spect_target.csv' saved successfully.",
 "File 'C:/Users/20192801/Documents/deep_learning_music_project\\onsets_ISMIR_2012\\csv_large_label\\ah_development_guitar_Guitar_Licks_07-06_start_end_spect_target.csv' saved successfully.",
 "File 'C:/Users/20192801/Documents/deep_learning_music_project\\onsets_ISMIR_2012\\csv_large_label\\ah_development_guitar_Guitar_Licks_07-11_start_end_spect_target.csv' saved successfully.",
 "File 'C:/Users/2019280